----
<img src="../../../files/refinitiv.png" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery - OMM Stream - Market Price data via callback

This notebook demonstrates how to use the OMM Stream interface to request streaming Quote and Trade data

## Set the location of the configuration file
For ease of use, you can set various initialization parameters of the RD Library in the **_refinitiv-data.config.json_** configuration file - as described in the Quick Start -> Sessions example.

### One config file for the tutorials
As these tutorial Notebooks are categorised into sub-folders and to avoid the need for multiple config files, we will use the _RD_LIB_CONFIG_PATH_ environment variable to point to a single instance of the config file in the top-level ***Configuration*** folder.

Before proceeding, please **ensure you have entered your credentials** into the config file in the ***Configuration*** folder.

In [1]:
import os
os.environ["RD_LIB_CONFIG_PATH"] = "../../../Configuration"

In [2]:
from refinitiv.data.delivery import omm_stream
import refinitiv.data as rd
import datetime
import json

## Open the default session

To open the default session ensure you have a '*refinitiv-data.config.json*' in the ***Configuration*** directory, populated with your credentials and specified a 'default' session in the config file    


In [3]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7ff067e56f10 {name='workspace'}>

## Define a function to display events

In [4]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

## Create an Item Stream and register the event callbacks

In [5]:
stream = omm_stream.Definition(
    name="EUR=", 
    fields=['BID', 'ASK','OPEN_PRC','HST_CLOSE','TIMACT'], 
    domain='MarketPrice').get_stream()

# Refresh - the first full image we get back from the server
stream.on_refresh(lambda event, stream  : display_event("Refresh", event))
# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda event, stream : display_event("Update", event))
# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda event, stream : display_event("Status", event))
# Other errors
stream.on_error(lambda event, stream : display_event("Error", event))

## Open the OMM Stream
The **open()** call to open the OMM Stream is a synchronous one. This means the first event (either via on_refresh(), on_status() or on_error()) can occur before the open() method returns.   
However, if we use the **open_async()** asynchronous method instead, the first event callback will be after  open_async() returns.

In [6]:
# Send request to server and open stream
stream.open()
# We should receive the initial Refresh for the current field values
# followed by updates for the fields as and when they occur

----------------------------------------------------------
>>> Refresh event received at 17:51:55.032083
{
  "ID": 5,
  "Type": "Refresh",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "State": {
    "Stream": "Open",
    "Data": "Ok",
    "Text": "New Session Mounted"
  },
  "Qos": {
    "Timeliness": "Realtime",
    "Rate": "JitConflated"
  },
  "PermData": "AwD+Umw=",
  "SeqNumber": 9374,
  "Fields": {
    "BID": 1.0585,
    "ASK": 1.0589,
    "OPEN_PRC": 1.0525,
    "HST_CLOSE": 1.0525,
    "TIMACT": "15:51:00"
  }
}


<OpenState.Opened: 'Opened'>

----------------------------------------------------------
>>> Update event received at 17:51:55.082900
{
  "ID": 5,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "SeqNumber": 9390,
  "Fields": {
    "BID": 1.0585,
    "ASK": 1.0589,
    "TIMACT": "15:51:00"
  }
}
----------------------------------------------------------
>>> Update event received at 17:51:56.105043
{
  "ID": 5,
  "Type": "Update",
  "UpdateType": "Unspecified",
  "Key": {
    "Service": "IDN_RDFNTS_CF",
    "Name": "EUR="
  },
  "SeqNumber": 9454,
  "Fields": {
    "BID": 1.0585,
    "ASK": 1.0589,
    "TIMACT": "15:51:00"
  }
}


## Close Stream

In [ ]:
stream.close()

## Close the session

In [8]:
rd.close_session()